# CE49X – Lab 5: Bias–Variance Tradeoff using the Air Quality Dataset

**Course:** CE49X – Introduction to Computational Thinking and Data Science for Civil Engineers  
**Instructor:** Dr. Eyuphan Koç  
**Semester:** Fall 2025  

**Objective.** Explore the **bias–variance tradeoff** by fitting polynomial regression models (degrees 1–10) to predict **CO(GT)** from meteorological variables (**T, RH, AH**) in the UCI Air Quality dataset. You will quantify **training** and **testing** error, draw a **validation curve**, and discuss **underfitting → optimal → overfitting** behavior.

## 🎯 Learning Goals

By the end of this lab, you will be able to:
- Explain the **bias–variance tradeoff** in supervised learning.
- Implement **linear** and **polynomial regression** models in scikit-learn.
- Compute and compare **training** vs **testing** error metrics across model complexity.
- Draw and interpret a **validation curve** and label **underfitting**, **optimal**, and **overfitting** regions.
- Reflect on how **sensor noise** and **missing data** influence the tradeoff.

## Step 1 — Load and Prepare Data

**Tasks**
1. Load `AirQualityUCI.csv` with **pandas**. The dataset often uses `-200` to denote missing values.
2. Replace `-200` with `NaN` and handle missingness (we will **drop** rows with missing target and **impute** features with the **median** inside an ML pipeline to avoid leakage).
3. Select features and target:
   - Features: `['T', 'RH', 'AH']`
   - Target: `'CO(GT)'`
4. Perform a **70%/30%** train/test split (`random_state=42` for reproducibility).

In [ ]:
# ---- Step 1: Imports, configuration, and robust CSV loading ----
# We import the libraries we'll use throughout the lab.
# - numpy, pandas for data handling
# - matplotlib (and optionally seaborn) for plotting
# - scikit-learn for model building, preprocessing, and evaluation

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Make plots a bit nicer
sns.set(context="notebook", style="whitegrid")

# Path to the dataset (already uploaded alongside this notebook)
from pathlib import Path

candidate_paths = [
    Path('AirQualityUCI.csv'),
    Path('girdiler') / 'AirQualityUCI.csv',
]
for candidate in candidate_paths:
    if candidate.exists():
        dataset_path = candidate.resolve()
        break
else:
    raise FileNotFoundError("AirQualityUCI.csv not found. Place it next to the notebook or inside a 'girdiler/' folder.")

print(f"Loading dataset from: {dataset_path}")

# Robust CSV loading:
# Many copies of this dataset use ';' as delimiter and ',' as decimal.
# We'll try that first; if it fails, fall back to default CSV.
def load_air_quality_csv(path):
    path = str(path)
    try:
        df_try = pd.read_csv(path, sep=';', decimal=',', na_values=[-200])
        # If it loaded but numeric columns are still strings, try coercing below.
        return df_try
    except Exception:
        # Fallback if the file uses default comma delimiter
        return pd.read_csv(path, na_values=[-200])

df_raw = load_air_quality_csv(dataset_path)

# Coerce numeric columns to numeric where possible (non-convertible stay as NaN).
for col in df_raw.columns:
    df_raw[col] = pd.to_numeric(df_raw[col], errors='ignore')  # keep non-numeric (e.g., Date/Time) unchanged

# For safety, replace any lingering sentinel -200 with NaN
df_raw = df_raw.replace(-200, np.nan)

# Display the head to confirm columns exist (T, RH, AH, CO(GT))
display(df_raw.head())
print("Columns:", list(df_raw.columns))

In [ ]:
# ---- Step 1: Feature selection and train/test split ----
# We select the features and target required by the lab.
features = ['T', 'RH', 'AH']
target = 'CO(GT)'

# Keep only the necessary columns (ignore date/time and other sensors)
missing_cols = [c for c in features + [target] if c not in df_raw.columns]
if missing_cols:
    raise ValueError(f"Required column(s) not found: {missing_cols}. "
                     "Please check the CSV column names.")

df = df_raw[features + [target]].copy()

# Separate features (X) and target (y)
X = df[features]
y = df[target]

# Drop rows with missing target; we'll impute X inside the ML pipeline
mask = y.notna()
X = X[mask]
y = y[mask]

# Perform a 70%/30% train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

print(f"Train shape: X={X_train.shape}, y={y_train.shape}")
print(f"Test  shape: X={X_test.shape}, y={y_test.shape}")

## Step 2 — Fit Models of Increasing Complexity (Degrees 1–10)

We will build a **pipeline**: `SimpleImputer(median) → PolynomialFeatures(degree=d, include_bias=False) → LinearRegression()`

For each degree `d ∈ {1,2,...,10}`:
1. Fit the model on the **training** data.
2. Compute **training MSE** and **testing MSE** to quantify performance.

In [ ]:
# ---- Step 2: Train models and record errors ----
# We'll iterate degrees 1..10, fitting a pipeline each time and recording errors.
degrees = list(range(1, 11))
train_mse = []
test_mse = []

for d in degrees:
    # Build a pipeline to avoid data leakage (impute only using train)
    model = make_pipeline(
        SimpleImputer(strategy='median'),
        PolynomialFeatures(degree=d, include_bias=False),
        LinearRegression()
    )
    model.fit(X_train, y_train)

    # Predictions on training and testing sets
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Mean Squared Error (MSE)
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)

    train_mse.append(mse_train)
    test_mse.append(mse_test)

# Store results in a DataFrame for easy viewing
results_df = pd.DataFrame({
    "degree": degrees,
    "train_mse": train_mse,
    "test_mse": test_mse
})
display(results_df)

## Step 3 — Plot Validation Curve

We plot **polynomial degree** (x-axis) vs **error (MSE)** (y-axis) for both **training** and **testing** sets.
We then **label regions** of:
- **Underfitting** (left, simple models: high bias, both errors high),
- **Optimal** (near minimum test error),
- **Overfitting** (right, complex models: training error low, test error high).

In [ ]:
# ---- Step 3: Validation curve plot with labeled regions ----
# Determine best degree (minimum test MSE)
best_idx = int(np.argmin(test_mse))
best_degree = degrees[best_idx]
best_test_mse = test_mse[best_idx]

plt.figure(figsize=(8, 5))
plt.plot(degrees, train_mse, marker='o', label='Training MSE')
plt.plot(degrees, test_mse, marker='s', label='Testing MSE')
plt.axvline(best_degree, linestyle='--', alpha=0.7, label=f'Best degree = {best_degree}')

# Shade underfitting and overfitting zones
if best_degree > min(degrees):
    plt.axvspan(min(degrees)-0.5, best_degree-0.5, alpha=0.08, label='Underfitting zone')
if best_degree < max(degrees):
    plt.axvspan(best_degree+0.5, max(degrees)+0.5, alpha=0.08, label='Overfitting zone')

plt.title('Bias–Variance Tradeoff: Validation Curve (MSE vs Degree)')
plt.xlabel('Model Complexity (Polynomial Degree)')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()
plt.tight_layout()
plt.show()

print(f"Best degree by test MSE: {best_degree} (Test MSE = {best_test_mse:.4f})")

## Step 4 — Discussion (Write-up)

Use your results to address the following:

1. **Best Generalization:** Which polynomial degree achieved the **lowest testing MSE**? Does the validation curve support this choice?
2. **Error Trends:** How did **training** and **testing** errors change as degree increased? Identify the **underfitting**, **optimal**, and **overfitting** regions on your plot.
3. **Bias–Variance Behavior:** Explain how high **bias** (simple models) and high **variance** (complex models) appear in this dataset.
4. **Data Quality Effects:** Discuss how **sensor noise** and **missing values** (e.g., `-200`) may influence the tradeoff, especially the tendency to overfit noisy signals.

> Tip: Re-run the notebook if you change any settings to update the plot and the best degree.

### 🔥 Bonus (Optional): Cross-Validation

Instead of a single train/test split, use **k-fold cross-validation** to estimate generalization error more robustly.
We will compare mean CV MSE across degrees and see if the optimal degree changes.

In [ ]:
# ---- Bonus: Cross-validation comparison ----
# We'll do 5-fold CV with shuffling for stability.
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_mean_mse = []
cv_std_mse = []

for d in degrees:
    model = make_pipeline(
        SimpleImputer(strategy='median'),
        PolynomialFeatures(degree=d, include_bias=False),
        LinearRegression()
    )
    # cross_val_score returns NEGATIVE MSE when using 'neg_mean_squared_error'
    neg_mse_scores = cross_val_score(
        model, X, y, cv=kf, scoring='neg_mean_squared_error'
    )
    mse_scores = -neg_mse_scores
    cv_mean_mse.append(mse_scores.mean())
    cv_std_mse.append(mse_scores.std())

cv_df = pd.DataFrame({
    "degree": degrees,
    "cv_mean_mse": cv_mean_mse,
    "cv_std_mse": cv_std_mse
}).set_index("degree")

display(cv_df)

best_cv_degree = int(cv_df["cv_mean_mse"].idxmin())
print(f"Best degree by 5-fold CV (mean MSE): {best_cv_degree} "
      f"(Mean MSE = {cv_df.loc[best_cv_degree, 'cv_mean_mse']:.4f} ± {cv_df.loc[best_cv_degree, 'cv_std_mse']:.4f})")

# Optional: plot CV curve
plt.figure(figsize=(8, 5))
plt.errorbar(degrees, cv_mean_mse, yerr=cv_std_mse, marker='o', capsize=4, label='CV Mean ± SD (MSE)')
plt.axvline(best_cv_degree, linestyle='--', alpha=0.7, label=f'Best degree (CV) = {best_cv_degree}')
plt.title('Cross-Validation Curve: MSE vs Degree (5-fold)')
plt.xlabel('Model Complexity (Polynomial Degree)')
plt.ylabel('Mean Squared Error (MSE)')
plt.legend()
plt.tight_layout()
plt.show()

## ✅ Conclusion (Short Summary)

- **Underfitting** occurs at **low degrees**: both training and testing errors are relatively high because the model is too simple (high **bias**).
- **Overfitting** occurs at **high degrees**: training error becomes very low while testing error rises due to memorizing noise (high **variance**).
- The **optimal degree** typically lies **in between**, where the **testing MSE** reaches its minimum (the **sweet spot** on the validation curve).
- With environmental **sensor data**, noise and missing observations can encourage overfitting—careful **imputation** and **regularization / model selection** are essential.

In [ ]:
# (Optional) Quick summary printout for graders/instructors
# This cell prints the best degree by test MSE and shows a compact result table.

summary_df = results_df.copy()
summary_df["is_best_test"] = summary_df["degree"] == summary_df["test_mse"].idxmin() + 1 if hasattr(summary_df["test_mse"], "idxmin") else False
print("Best degree by test MSE:", int(np.argmin(test_mse) + 1 if len(test_mse) else -1))
display(summary_df)